In [1]:
# from transformers import AutoTokenizer, EsmForSequenceClassification
import torch
import pandas as pd
import esm

In [101]:
# Can be used
# model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t6_8M_UR50D")
model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()

In [3]:
# model has a base class nn.Module
print(esm.model.esm2.ESM2.__bases__)
# model

(<class 'torch.nn.modules.module.Module'>,)


In [102]:
from torch.nn import Identity
model.contact_head = Identity()
model.emb_layer_norm_after = Identity()
model.lm_head = Identity()
# model

In [5]:
df = pd.read_csv("/home/share/huadjyin/home/nikolamilicevic/pfam_subs/pfam.train.subs.csv")

In [80]:
seq_test = df['seq'][2] * 20

In [82]:
len(seq_test)

4060

In [90]:
4060*100*480*4 / (1024*1024)

743.408203125

In [91]:
# free / (1024*1024)
def check_gpu_mem(device):
    free, total = torch.cuda.mem_get_info(device)
    print(f"Free: {free / (1024*1024)} Total: {total / (1024*1024)}")

def get_tensor_size_mb(tens):
    sz = tens.element_size() * tens.nelement()
    sz /= (1024 * 1024)
    print(f"Size MiB: {sz}")


In [92]:
batch_converter = alphabet.get_batch_converter()
data = [
    # ("protein1", seq_test),
    ("protein2", seq_test) for _ in range(50)
    # ("prot1", "ADPALVLH"),
    # ("prot2", "ADPAL"),
    # ("prot3", "AD")
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

In [93]:
device = torch.device("cuda:2")
# alphabet.tok_to_idx
# '<cls>': 0,
#  '<pad>': 1,
#  '<eos>': 2,
#  '<unk>': 3,
# ...
# '.': 29,
#  '-': 30,
#  '<null_1>': 31,
#  '<mask>': 32}

In [99]:
# batch_tokens = batch_tokens.to(torch.int16)
batch_tokens

tensor([[ 0, 11, 13,  ...,  4,  4,  2],
        [ 0, 11, 13,  ...,  4,  4,  2],
        [ 0, 11, 13,  ...,  4,  4,  2],
        ...,
        [ 0, 11, 13,  ...,  4,  4,  2],
        [ 0, 11, 13,  ...,  4,  4,  2],
        [ 0, 11, 13,  ...,  4,  4,  2]])

In [100]:
# torch.cuda.is_available()
print(batch_tokens.device)
# torch.cuda.device_count()
print(batch_tokens[0].dtype)
check_gpu_mem(device)
batch_tokens = batch_tokens.to(device)

cpu
torch.int64
Free: 37568.5625 Total: 40354.4375


In [14]:
# torch.cuda.empty_cache()
# check_gpu_mem(device)
# get_tensor_size_mb(batch_tokens)
# batch_tokens.shape
check_gpu_mem(device)
get_tensor_size_mb(batch_tokens)

Free: 38489.5625 Total: 40354.4375
Size MiB: 0.309906005859375


In [103]:
# tot = torch.cuda.memory_reserved(device)
# free, total = torch.cuda.mem_get_info(device)
# tot / (1024*1024)
batch_tokens.requires_grad

False

In [104]:
# x = torch.randn(3,3)
# x = x.to(device)
# x.device
# model = model.to(device)
torch.cuda.max_memory_allocated()

0

In [19]:
import gc 
del batch_tokens
gc.collect()
torch.cuda.empty_cache()

In [75]:
def count_model_size_mb(model):
    return sum(p.nelement() * p.element_size() for p in model.parameters()) / (1024 * 1024)

def count_parameters(model):
    return sum(p.nelement() for p in model.parameters())

In [78]:
count_parameters(model)

33501394

In [71]:
# next(model.parameters()).dtype
# 35*1e6*4/(1024*1024*1024)
model.to(torch.bfloat16)
next(model.parameters())

Parameter containing:
tensor([[ 0.0525, -0.2617, -0.0292,  ..., -0.0635, -0.0179,  0.0432],
        [-0.1777, -0.5977,  0.1943,  ...,  0.3672,  0.1357,  0.0131],
        [ 0.0981, -0.2754,  0.0186,  ..., -0.0469,  0.1670, -0.0327],
        ...,
        [ 0.0022, -0.0425,  0.0080,  ...,  0.0479, -0.0464, -0.0066],
        [-0.0014, -0.0260,  0.0347,  ...,  0.0342, -0.0659, -0.0422],
        [-0.1807, -0.5977,  0.1934,  ...,  0.3711,  0.1455,  0.0138]],
       dtype=torch.bfloat16, requires_grad=True)

In [72]:
count_parameters(model)

63.89883804321289

In [105]:
model = model.to(device)

In [108]:
check_gpu_mem(device)

Free: 4738.5625 Total: 40354.4375


In [18]:
model = model.to(device)
model.eval()
with torch.no_grad():
    res = model(batch_tokens)

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.29 GiB (GPU 1; 39.41 GiB total capacity; 13.40 GiB already allocated; 12.11 GiB free; 25.13 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [43]:
res['logits'].shape

torch.Size([1, 4062, 480])

In [21]:
res

{'logits': tensor([[[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.0620],
          [ 4.4105,  2.9949, -1.7162,  ...,  4.4722, -0.3008, -2.9338],
          [-0.7679, -2.4428, -1.0123,  ..., -1.6110,  2.6296,  0.2919],
          ...,
          [-1.7249, -1.8000,  4.2837,  ...,  1.0689, -0.8234, -5.0085],
          [-0.1915, -0.7245,  2.4475,  ..., -0.7194,  0.2490, -4.1189],
          [-1.1293,  1.0655,  1.3112,  ...,  1.8803, -6.6340, -4.0444]],
 
         [[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.0620],
          [ 4.4105,  2.9949, -1.7162,  ...,  4.4722, -0.3008, -2.9338],
          [-0.7679, -2.4428, -1.0123,  ..., -1.6110,  2.6296,  0.2919],
          ...,
          [-1.7249, -1.8000,  4.2837,  ...,  1.0689, -0.8234, -5.0085],
          [-0.1915, -0.7245,  2.4475,  ..., -0.7194,  0.2490, -4.1189],
          [-1.1293,  1.0655,  1.3112,  ...,  1.8803, -6.6340, -4.0444]]]),
 'representations': {6: tensor([[[ 2.5057,  6.8665,  1.0299,  ...,  8.5440, -1.4410, -7.06